In [56]:
# The goal of this kernel is to demonstrate that LightGBM can have predictive
# performance in line with that of a logistic regression. The theory is that
# labeling is being driven by a few keywords that can be picked up by trees.
#
# With some careful tuning, patience with runtimes, and additional feature
# engineering, this kernel can be tuned to slightly exceed the best
# logistic regression. Best of all, the two approaches (LR and LGB) blend
# well together.
#
# Hopefully, with some work, this could be a good addition to your ensemble.

import gc
import pandas as pd

from scipy.sparse import csr_matrix, hstack


from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel

from sklearn.linear_model import LogisticRegression
import lightgbm as lgb



In [57]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('../input/internal_train.csv').fillna(' ')
test = pd.read_csv('../input/internal_test.csv').fillna(' ')
print('Loaded')

train_text = train['comment_text']
test_text = test['comment_text']

Loaded


In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 2),
    max_features=50000)
train_word_features = word_vectorizer.fit_transform(train_text)
print('Word TFIDF 1/2')
test_word_features = word_vectorizer.transform(test_text)
print('Word TFIDF 2/2')

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)

train_char_features = char_vectorizer.fit_transform(train_text)
print('Char Count 1/2')
test_char_features = char_vectorizer.transform(test_text)
print('Char Count 2/2')

train_features = hstack([ train_char_features, train_word_features])
print('HStack 1/2')
test_features = hstack([ test_char_features, test_word_features])
print('HStack 2/2')

Word TFIDF 1/2
Word TFIDF 2/2
Char Count 1/2
Char Count 2/2
HStack 1/2
HStack 2/2


In [59]:
from sklearn.feature_extraction.text import CountVectorizer

word_vectorizer = CountVectorizer(
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 2),
    max_features=50000)

train_word_features = word_vectorizer.fit_transform(train_text)
print('Word Count 1/2')
test_word_features = word_vectorizer.transform(test_text)
print('Word Count 2/2')

char_vectorizer = CountVectorizer(
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)
train_char_features = char_vectorizer.fit_transform(train_text)
print('Char Count 1/2')
test_char_features = char_vectorizer.transform(test_text)
print('Char Count 2/2')



Word Count 1/2
Word Count 2/2
Char Count 1/2
Char Count 2/2


In [60]:
print(train_features.shape)
print(test_features.shape)

(79785, 100000)
(79786, 100000)


In [61]:
train_features = hstack([ train_features, train_char_features, train_word_features])
print('HStack 1/2')
test_features = hstack([ test_features, test_char_features, test_word_features])
print('HStack 2/2')

HStack 1/2
HStack 2/2


In [62]:
submission = pd.DataFrame.from_dict({'id': test['id']})

train.drop('comment_text', axis=1, inplace=True)
del test
del train_text
del test_text
del train_char_features
del test_char_features
del train_word_features
del test_word_features
gc.collect()

220

In [63]:
all_train_sparse_matrix = {}
all_test_sparse_matrix = {}

for class_name in class_names:
    print(class_name)
    train_target = train[class_name]
    model = LogisticRegression(solver='sag')
    sfm = SelectFromModel(model)
    
    print('Train: before feature selection\'s shape: ', train_features.shape)
    train_sparse_matrix = sfm.fit_transform(train_features, train_target)
    print('After', train_sparse_matrix.shape)
    #train_sparse_matrix, valid_sparse_matrix, y_train, y_valid = train_test_split(train_sparse_matrix, train_target, test_size=0.05, random_state=144)
    print('Test: before feature selection\'s shape:', test_features.shape)
    test_sparse_matrix = sfm.transform(test_features)
    print('After', test_sparse_matrix.shape)
    
    all_train_sparse_matrix[class_name] = train_sparse_matrix
    all_test_sparse_matrix[class_name] = test_sparse_matrix

    

toxic
Train: before feature selection's shape:  (79785, 200000)


/home/willy/ENV/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


After (79785, 30416)
Test: before feature selection's shape: (79786, 200000)
After (79786, 30416)
severe_toxic
Train: before feature selection's shape:  (79785, 200000)


/home/willy/ENV/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


After (79785, 28462)
Test: before feature selection's shape: (79786, 200000)
After (79786, 28462)
obscene
Train: before feature selection's shape:  (79785, 200000)


...

After (79785, 27630)
Test: before feature selection's shape: (79786, 200000)
After (79786, 27630)
insult
Train: before feature selection's shape:  (79785, 200000)


/home/willy/ENV/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


After (79785, 28616)
Test: before feature selection's shape: (79786, 200000)
After (79786, 28616)
identity_hate
Train: before feature selection's shape:  (79785, 200000)


/home/willy/ENV/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


After (79785, 27381)
Test: before feature selection's shape: (79786, 200000)
After (79786, 27381)


In [65]:
import numpy as np
params = {'learning_rate': 0.2,
              'application': 'binary',
              'num_leaves': 31,
              'verbosity': -1,
              'metric': 'auc',
              'data_random_seed': 2,
              'bagging_fraction': 0.8,
              'feature_fraction': 0.6,
              'nthread': 4,
              'lambda_l1': 1,
              'lambda_l2': 1}

rounds_lookup = {'toxic': 140,
                 'severe_toxic': 50,
                 'obscene': 80,
                 'threat': 80,
                 'insult': 70,
                 'identity_hate': 80}

final_val_score = []
for class_name in class_names:
    print(class_name)
    train_sparse_matrix = all_train_sparse_matrix[class_name].astype(np.float32)
    test_sparse_matrix = all_test_sparse_matrix[class_name].astype(np.float32)
    
    y_train = train[class_name]
    d_train = lgb.Dataset(train_sparse_matrix, label=y_train.astype(np.float32))
    
    res = lgb.cv(params,
                 train_set=d_train,
                 num_boost_round=rounds_lookup[class_name],
                 verbose_eval=10)
    
    rounds = np.argmax(res['auc-mean'])
    final_val_score.append(res['auc-mean'][rounds])
    print(rounds)
    model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=rounds,
                      verbose_eval=10)
    
    submission[class_name] = model.predict(test_sparse_matrix)

    print(final_val_score)
    print('final score:', np.mean(final_val_score))
submission.to_csv('lgb_submission.csv', index=False)

toxic
[10]	cv_agg's auc: 0.934606 + 0.00258363
[20]	cv_agg's auc: 0.950412 + 0.00362118
[30]	cv_agg's auc: 0.959527 + 0.00312989
[40]	cv_agg's auc: 0.96463 + 0.00271708
[50]	cv_agg's auc: 0.966909 + 0.00218623
[60]	cv_agg's auc: 0.968307 + 0.00229625
[70]	cv_agg's auc: 0.968768 + 0.00217757
[80]	cv_agg's auc: 0.969036 + 0.00203142
[90]	cv_agg's auc: 0.969091 + 0.00188448
[100]	cv_agg's auc: 0.96939 + 0.0018902
[110]	cv_agg's auc: 0.969484 + 0.00182103
[120]	cv_agg's auc: 0.96937 + 0.00175764
[130]	cv_agg's auc: 0.969392 + 0.00170366
[140]	cv_agg's auc: 0.969325 + 0.00168527
111
[0.96951737056053966]
final score: 0.969517370561
severe_toxic
[10]	cv_agg's auc: 0.94653 + 0.0210525
[20]	cv_agg's auc: 0.978567 + 0.00553647
[30]	cv_agg's auc: 0.980923 + 0.00520922
[40]	cv_agg's auc: 0.981548 + 0.0064798
[50]	cv_agg's auc: 0.981743 + 0.00565448
45
[0.96951737056053966, 0.98211140695457044]
final score: 0.975814388758
obscene
[10]	cv_agg's auc: 0.974065 + 0.00235525
[20]	cv_agg's auc: 0.984387